In [1]:
import pandas as p
import os as o
import numpy as np
import re as r 

# READ DATA

In [2]:
o.chdir(r"D:\fledging\projects\mins\Data\CSV\20190613")

## LISTING

In [30]:
listingFile="eBay_Listings_2019-06-12.csv"
salesFile="eBay_Market_Sales_2019_06_13.csv"

In [6]:
#top10=p.DataFrame()
listing=p.read_csv(open(listingFile, 'rb'),nrows=1)
cols = [col for col in listing.columns if col not in ['type','capacity','status','rawprice','currency','terms']]
print(cols)
colDtypes =['str','str','str','str','str','str', 'str']
parse_dates = ['datefirst', 'datelast']

dtypes=dict(zip(cols,colDtypes))
# dtypes

['id', 'cat', 'title', 'seller', 'price', 'datefirst', 'datelast']


In [8]:
listing=p.read_csv(open(listingFile, 'rb'), usecols=cols,dtype=dtypes,parse_dates=parse_dates,infer_datetime_format=True)
#listing.head()
#listing.head()
listing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8446 entries, 0 to 8445
Data columns (total 7 columns):
id           8446 non-null object
cat          8446 non-null object
title        7256 non-null object
seller       7257 non-null object
price        7255 non-null object
datefirst    8446 non-null datetime64[ns]
datelast     8446 non-null datetime64[ns]
dtypes: datetime64[ns](2), object(5)
memory usage: 462.0+ KB


In [9]:
listing.dropna(subset=['price'],inplace=True)
listing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7255 entries, 0 to 8445
Data columns (total 7 columns):
id           7255 non-null object
cat          7255 non-null object
title        7255 non-null object
seller       7255 non-null object
price        7255 non-null object
datefirst    7255 non-null datetime64[ns]
datelast     7255 non-null datetime64[ns]
dtypes: datetime64[ns](2), object(5)
memory usage: 453.4+ KB


In [10]:
#CONVERT TITLE TO UPPER
listing['title']=listing['title'].apply(lambda x: x.upper())
#REMOVE CAT LEADING ZEROS 
listing['cat']=listing['cat'].apply(lambda x: x.lstrip("00"))
listing.head()


,id,cat,title,seller,price,datefirst,datelast
0,192788976703,1,"MACBOOK AIR 13"" A1369 2010 TOSHIBA 256GB SSD ...",maduier,65.00,2019-02-08 00:16:00,2019-05-22 23:19:00
2,202591255356,1,256GB SSD SOLID STATE HARD DRIVE FOR MACBOOK ...,bluedevil2017,59.99,2019-02-08 00:16:00,2019-05-03 21:33:00
4,143109036841,1,APPLE GENUINE 256GB SSD FOR APPLE MACBOOK AIR...,electstore4you,63.98,2019-02-08 00:16:00,2019-05-15 23:57:00
6,263189887482,1,NEW 256GB SSD FOR 2010 2011 APPLE MACBOOK AIR...,hatssd,69.00,2019-02-08 00:16:00,2019-06-12 23:18:00
7,283360223952,1,SAMSUNG 256 GB SSD FOR 2010 & 2011 MACBOOK AI...,protechrestore,57.53,2019-02-08 00:16:00,2019-05-22 23:19:00


In [11]:
#CONVERT PRICE TO FLOAT
listing.loc[:,'price']=listing.loc[:,'price'].apply(lambda x:r.sub("[^0-9.]", "",x))
listing['price']=listing['price'].astype(float)

In [12]:
listing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7255 entries, 0 to 8445
Data columns (total 7 columns):
id           7255 non-null object
cat          7255 non-null object
title        7255 non-null object
seller       7255 non-null object
price        7255 non-null float64
datefirst    7255 non-null datetime64[ns]
datelast     7255 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(1), object(4)
memory usage: 453.4+ KB


In [13]:
def  extract_capacity(patternstr,inputstr):
    try:
        cleanstr=inputstr.replace(" ", "*")
        match=r.search(patternstr,cleanstr,flags=0)
        start = match.start()
        end= match.end()
        if inputstr[int(start):int(end)]=="TB":    
            cap=inputstr[int(start-2):int(start)]+" "+inputstr[int(start):int(end)]
            return cap
        else:
            cap=inputstr[int(start-3):int(start)] +" "+inputstr[int(start):int(end)]
            return(cap)
    except (TypeError,AttributeError):
        return "NA"

In [14]:
#GET CAPACITY
listing['capacity']=listing["title"].apply(lambda x: extract_capacity("GB|TB",x))
listing.head()

,id,cat,title,seller,price,datefirst,datelast,capacity
0,192788976703,1,"MACBOOK AIR 13"" A1369 2010 TOSHIBA 256GB SSD ...",maduier,65.00,2019-02-08 00:16:00,2019-05-22 23:19:00,256 GB
2,202591255356,1,256GB SSD SOLID STATE HARD DRIVE FOR MACBOOK ...,bluedevil2017,59.99,2019-02-08 00:16:00,2019-05-03 21:33:00,256 GB
4,143109036841,1,APPLE GENUINE 256GB SSD FOR APPLE MACBOOK AIR...,electstore4you,63.98,2019-02-08 00:16:00,2019-05-15 23:57:00,256 GB
6,263189887482,1,NEW 256GB SSD FOR 2010 2011 APPLE MACBOOK AIR...,hatssd,69.00,2019-02-08 00:16:00,2019-06-12 23:18:00,256 GB
7,283360223952,1,SAMSUNG 256 GB SSD FOR 2010 & 2011 MACBOOK AI...,protechrestore,57.53,2019-02-08 00:16:00,2019-05-22 23:19:00,56 GB


In [15]:
print(listing
        .groupby(['capacity'])
        .agg(
                {
                    'capacity':"count"
                }
            ).head()
            )


          capacity
capacity          
 (8 GB           1
 1 TB          602
 16 GB          36
 2 TB           72
 30 GB           1


In [16]:
listing.loc[listing['capacity']=="NA"]

,id,cat,title,seller,price,datefirst,datelast,capacity
114,273657419443,1,QNINE SSD ADAPTER CONVERTER CARD FOR 2010 201...,jd68000,4.99,2019-02-08 00:16:00,2019-05-22 23:19:00,NA
131,123359327368,1,QNINE SSD ENCLOSURE FOR MACBOOK AIR 2010 2011...,marxseller,30.29,2019-02-08 00:16:00,2019-06-12 23:18:00,NA
134,113566009438,1,SHINESTAR M.2 NGFF SSD TO A1369 A1370 ADAPTER...,patrickatozwarehouse1,13.86,2019-02-08 00:16:00,2019-03-28 10:18:00,NA
136,312232615803,1,SHINESTAR M.2 NGFF SSD TO A1369 A1370 ADAPTER...,great_bargain_store,13.93,2019-02-08 00:16:00,2019-03-28 10:18:00,NA
347,192815932169,4,APPLE SAMSUNG 512 SSD FLASH STORAGE FOR MACBO...,geo-tec,204.06,2019-02-08 00:16:00,2019-05-03 21:33:00,NA
430,254038821430,4,"MACBOOK PRO 15"" A1398 13"" A1502 MZ-KPU1T0T/0A...",starpro2015,290.69,2019-02-08 00:16:00,2019-06-12 23:18:00,NA
431,254087372312,4,"SD6PQ4M-256G-1021 PCIE SSD MACBOOK AIR 2013, ...",starpro2015,75.99,2019-02-08 00:16:00,2019-05-30 12:59:00,NA
571,192801492334,4,SINTECH M.2 M-KEY PCIE NVME SSD 2013-2016 APP...,todbretton,19.99,2019-02-08 00:16:00,2019-06-12 23:18:00,NA
633,143113219314,4,MZ-DPC256T/0A2 APPLE SAMSUNG 256G SSD MACBOOK...,northbaysell,89.00,2019-02-08 00:16:00,2019-05-15 23:57:00,NA
652,173639160637,4,MZ-DPC256T/0A2 APPLE SAMSUNG 256G SSD MACBOOK...,the-it-mart,98.99,2019-02-08 00:16:00,2019-03-07 14:21:00,NA


In [17]:
listing.groupby(['price','cat']).ngroups

2429

## LOOKUP TABLE

In [18]:
### TAKEN FROM MARKET HISTORY LISTING SHEET
file=r"D:\fledging\Reports\marketAnalysis\MarketHistory_lookups.csv"
catLkp=p.read_csv(open(file, 'rb'),dtype={"id":object,"cat":object})

In [19]:
catLkp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7424 entries, 0 to 7423
Data columns (total 2 columns):
id     7424 non-null object
cat    7424 non-null object
dtypes: object(2)
memory usage: 116.1+ KB


# SALES DATA

In [43]:
sales=p.read_csv(open(salesFile, 'rb'),nrows=1)
cols = [col for col in sales.columns if col not in ['col3']]
# print(sales.columns)
colDtypes =['str']*len(cols)
parse_dates = ['date', 'datefirst']
dtypes=dict(zip(cols,colDtypes))
dtypes
sales=p.read_csv(open(salesFile, 'rb'), usecols=cols,dtype=dtypes,parse_dates=parse_dates,infer_datetime_format=True)
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11235 entries, 0 to 11234
Data columns (total 6 columns):
id           11235 non-null object
price        11235 non-null object
quantity     11235 non-null object
date         11235 non-null datetime64[ns]
datefirst    11235 non-null datetime64[ns]
timezone     11235 non-null object
dtypes: datetime64[ns](2), object(4)
memory usage: 526.7+ KB


In [45]:
start_date='01-Jun-19'
end_date='30-Jun-19'
filter_date = (sales['date'] >= start_date) & (sales['date'] <= end_date)
sales=sales.loc[filter_date]
sales=sales.reset_index(drop=True)
sales.head()

,id,price,quantity,date,datefirst,timezone
0,112152782112,US $43.20,1,2019-06-12,2019-06-13 20:13:00,EDT
1,112152782112,US $43.20,1,2019-06-05,2019-06-13 19:15:11,EDT
2,322555440929,US $20.30,1,2019-06-03,2019-06-13 08:33:27,EDT
3,233047715110,Accepted,2,2019-06-04,2019-06-13 16:36:48,EDT
4,263246991178,Accepted,1,2019-06-02,2019-06-13 08:50:46,EDT


In [46]:
sales=sales.merge(catLkp,on=["id"],how="left")
sales.info()
#sales.reset_index(drop=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 638 entries, 0 to 637
Data columns (total 7 columns):
id           638 non-null object
price        638 non-null object
quantity     638 non-null object
date         638 non-null datetime64[ns]
datefirst    638 non-null datetime64[ns]
timezone     638 non-null object
cat          519 non-null object
dtypes: datetime64[ns](2), object(5)
memory usage: 39.9+ KB


In [47]:
filter_cat=(sales["cat"]!="-1") & (sales["cat"]!="external")
sales=sales.loc[filter_cat,:]

In [48]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 559 entries, 0 to 637
Data columns (total 7 columns):
id           559 non-null object
price        559 non-null object
quantity     559 non-null object
date         559 non-null datetime64[ns]
datefirst    559 non-null datetime64[ns]
timezone     559 non-null object
cat          440 non-null object
dtypes: datetime64[ns](2), object(5)
memory usage: 34.9+ KB


In [49]:
sales=sales.merge(listing.loc[:,['id','seller','capacity','price']],on=["id"],how="left",suffixes=["","li"])

In [50]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 559 entries, 0 to 558
Data columns (total 10 columns):
id           559 non-null object
price        559 non-null object
quantity     559 non-null object
date         559 non-null datetime64[ns]
datefirst    559 non-null datetime64[ns]
timezone     559 non-null object
cat          440 non-null object
seller       559 non-null object
capacity     559 non-null object
priceli      559 non-null float64
dtypes: datetime64[ns](2), float64(1), object(7)
memory usage: 48.0+ KB


In [51]:
#CONVERT QUANTITY TO INT
sales.loc[:,'quantity']=sales.loc[:,'quantity'].astype(np.int32)

In [52]:
def currreg(str):
    str=r.sub(",", "", str)
    m=r.search("(\w{1,3}).{0,3}?(\d+,*\d+,*\.?\d{1,2})",str)
    try:
        #print(str+"-this is not going nan")
        return [m.group(1),float(m.group(2))]
    except:
        #print(str+"-this is going nan")
        return [np.nan,np.nan]
    
sales['flagCurr']=sales.loc[:,'price'].apply(lambda x:currreg(x)[0])
sales['priceClean']=sales.loc[:,'price'].apply(lambda x:currreg(x)[1])

In [53]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 559 entries, 0 to 558
Data columns (total 12 columns):
id            559 non-null object
price         559 non-null object
quantity      559 non-null int32
date          559 non-null datetime64[ns]
datefirst     559 non-null datetime64[ns]
timezone      559 non-null object
cat           440 non-null object
seller        559 non-null object
capacity      559 non-null object
priceli       559 non-null float64
flagCurr      460 non-null object
priceClean    460 non-null float64
dtypes: datetime64[ns](2), float64(2), int32(1), object(7)
memory usage: 54.6+ KB


In [54]:
#"ACCEPTED" DATA 
#sales.loc[sales['pricesl'].isin(["Accepted","Sold as a special offer"])].count()
sales.loc[sales['priceClean'].isnull()].count()

id            99
price         99
quantity      99
date          99
datefirst     99
timezone      99
cat           81
seller        99
capacity      99
priceli       99
flagCurr       0
priceClean     0
dtype: int64

In [55]:
#SPLIT DATA TO FLOATS
salesUnclean=sales.loc[sales['priceClean'].isnull()]
salesClean=sales.loc[~(sales['priceClean'].isnull())]
salesUnclean=salesUnclean.reset_index(drop=True)
salesClean=salesClean.reset_index(drop=True)
salesClean.info()
salesUnclean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460 entries, 0 to 459
Data columns (total 12 columns):
id            460 non-null object
price         460 non-null object
quantity      460 non-null int32
date          460 non-null datetime64[ns]
datefirst     460 non-null datetime64[ns]
timezone      460 non-null object
cat           359 non-null object
seller        460 non-null object
capacity      460 non-null object
priceli       460 non-null float64
flagCurr      460 non-null object
priceClean    460 non-null float64
dtypes: datetime64[ns](2), float64(2), int32(1), object(7)
memory usage: 41.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 12 columns):
id            99 non-null object
price         99 non-null object
quantity      99 non-null int32
date          99 non-null datetime64[ns]
datefirst     99 non-null datetime64[ns]
timezone      99 non-null object
cat           81 non-null object
seller        99 non-null object
capaci

In [56]:
discount=1
salesUnclean['priceAdj']=salesUnclean['priceli']*discount

In [57]:
salesUnclean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 13 columns):
id            99 non-null object
price         99 non-null object
quantity      99 non-null int32
date          99 non-null datetime64[ns]
datefirst     99 non-null datetime64[ns]
timezone      99 non-null object
cat           81 non-null object
seller        99 non-null object
capacity      99 non-null object
priceli       99 non-null float64
flagCurr      0 non-null object
priceClean    0 non-null float64
priceAdj      99 non-null float64
dtypes: datetime64[ns](2), float64(3), int32(1), object(7)
memory usage: 9.7+ KB


In [58]:
#salesClean=salesClean.rename(columns={"price":"priceUnclean", "priceClean":"price"})
salesUnclean.head()

,id,price,quantity,date,datefirst,timezone,cat,seller,capacity,priceli,flagCurr,priceClean,priceAdj
0,233047715110,Accepted,2,2019-06-04,2019-06-13 16:36:48,EDT,1,jjtpa11,128 GB,41.99,NaN,NaN,41.99
1,263246991178,Accepted,1,2019-06-02,2019-06-13 08:50:46,EDT,1,hatssd,256 GB,69.00,NaN,NaN,69.00
2,263242706826,Accepted,1,2019-06-06,2019-06-13 22:50:07,EDT,2,hatssd,512 GB,129.99,NaN,NaN,129.99
3,263114845435,Accepted,1,2019-06-12,2019-06-13 10:51:44,EDT,2,hatssd,256 GB,79.99,NaN,NaN,79.99
4,263246989168,Accepted,1,2019-06-07,2019-06-13 01:07:53,EDT,2,hatssd,256 GB,79.99,NaN,NaN,79.99


In [59]:
salesClean.head()
# N=len(salesReal['price'])-1
# for i in range(0,N):
#     try:
#         sales['price']=sales["price"].apply(lambda x:float(x))
#     except:
#         print("error","on line",i)
# sales['price']=sales["price"].apply(lambda x:r.sub("[^0-9.]","",x))
# #sales["quantity"].apply(lambda x:r.sub("[^0-9.]", "",x))

# N=len(sales['price'])-1
# for i in range(0,N):
#     try:
#         sales['price']=sales["price"].apply(lambda x:float(x))
#     except:
#         print("error","on line",i)


,id,price,quantity,date,datefirst,timezone,cat,seller,capacity,priceli,flagCurr,priceClean
0,112152782112,US $43.20,1,2019-06-12,2019-06-13 20:13:00,EDT,1,tekdep,128 GB,43.20,US,43.20
1,112152782112,US $43.20,1,2019-06-05,2019-06-13 19:15:11,EDT,1,tekdep,128 GB,43.20,US,43.20
2,322555440929,US $20.30,1,2019-06-03,2019-06-13 08:33:27,EDT,1,electro_bego,64 GB,20.30,US,20.30
3,254065792421,US $56.24,1,2019-06-11,2019-06-13 12:57:14,EDT,1,emall365,240 GB,56.24,US,56.24
4,233012352119,US $29.99,1,2019-06-03,2019-06-13 11:56:00,EDT,1,drivesupply,64 GB,29.99,US,29.99


In [60]:
#CURRENCY RATE    
rates={"US":1,"AU":0.69,"C":0.74,"EUR":1.12,"GBP":1.26}

salesClean['rate']=salesClean['flagCurr'].apply(lambda x: rates[x])
salesClean['priceAdj']=salesClean['priceClean']*salesClean['rate']
salesClean.loc[salesClean['flagCurr']=="C"].head()


# def currRate(row):
#     for curr, rate in rates.items():
#         print(rates.items())
#         if row["flagCurr"] == curr:
#             print(row["flagCurr"])
#             return row["priceClean"]*rate
#         else:
#             print(row["flagCurr"])
#             return row["priceClean"]

#salesClean['priceAdj']=salesClean.apply(currRate,axis=1)

,id,price,quantity,date,datefirst,timezone,cat,seller,capacity,priceli,flagCurr,priceClean,rate,priceAdj
258,323781881196,C $88.88,1,2019-06-01,2019-06-13 15:47:17,EDT,2,szhdd,256 GB,88.88,C,88.88,0.74,65.7712
259,323781881196,C $88.88,1,2019-06-01,2019-06-13 07:12:47,EDT,2,szhdd,256 GB,88.88,C,88.88,0.74,65.7712
260,223486724464,C $62.88,1,2019-06-02,2019-06-13 05:11:35,EDT,2,szhdd,128 GB,62.88,C,62.88,0.74,46.5312
265,323781835383,C $39.88,1,2019-06-02,2019-06-13 13:21:22,EDT,2,szhdd,64 GB,39.88,C,39.88,0.74,29.5112
287,183791793112,C $135.00,1,2019-06-07,2019-06-13 13:31:24,EDT,4,1101montreal,256 GB,135.00,C,135.00,0.74,99.9000


In [159]:
#salesClean=salesClean.rename(columns={"price":"priceUnclean", "priceAdj":"price"})

In [61]:
salesUnclean.info()
salesClean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 13 columns):
id            99 non-null object
price         99 non-null object
quantity      99 non-null int32
date          99 non-null datetime64[ns]
datefirst     99 non-null datetime64[ns]
timezone      99 non-null object
cat           81 non-null object
seller        99 non-null object
capacity      99 non-null object
priceli       99 non-null float64
flagCurr      0 non-null object
priceClean    0 non-null float64
priceAdj      99 non-null float64
dtypes: datetime64[ns](2), float64(3), int32(1), object(7)
memory usage: 9.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460 entries, 0 to 459
Data columns (total 14 columns):
id            460 non-null object
price         460 non-null object
quantity      460 non-null int32
date          460 non-null datetime64[ns]
datefirst     460 non-null datetime64[ns]
timezone      460 non-null object
cat           359 non-null object
seller       

In [62]:
cols=salesClean.columns

In [63]:
sales2=p.concat([salesClean.loc[:,cols],salesUnclean.loc[:,cols]])

d:\fledging\projects\mins\env\projenv\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [64]:
sales2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 559 entries, 0 to 98
Data columns (total 14 columns):
id            559 non-null object
price         559 non-null object
quantity      559 non-null int32
date          559 non-null datetime64[ns]
datefirst     559 non-null datetime64[ns]
timezone      559 non-null object
cat           440 non-null object
seller        559 non-null object
capacity      559 non-null object
priceli       559 non-null float64
flagCurr      460 non-null object
priceClean    460 non-null float64
rate          460 non-null float64
priceAdj      559 non-null float64
dtypes: datetime64[ns](2), float64(4), int32(1), object(7)
memory usage: 63.3+ KB


In [65]:
sales2["revenue"]=sales2["priceAdj"]*sales2["quantity"]

In [66]:
sales2.loc[:,"revenue"].agg("sum")

112565.89090000001

In [67]:
df_curr=sales2

In [68]:
df_curr.loc[df_curr["priceAdj"]==0,:]

,id,price,quantity,date,datefirst,timezone,cat,seller,capacity,priceli,flagCurr,priceClean,rate,priceAdj,revenue


In [69]:
df_curr.loc[df_curr["id"]=="263785699614",:]

,id,price,quantity,date,datefirst,timezone,cat,seller,capacity,priceli,flagCurr,priceClean,rate,priceAdj,revenue


In [70]:
df_curr.loc[df_curr["seller"]=="hatssd",:]

,id,price,quantity,date,datefirst,timezone,cat,seller,capacity,priceli,flagCurr,priceClean,rate,priceAdj,revenue
6,263242706826,US $129.99,1,2019-06-05,2019-06-13 13:29:11,EDT,2,hatssd,512 GB,129.99,US,129.99,1.0,129.99,129.99
7,263242706826,US $139.99,1,2019-06-04,2019-06-13 11:45:34,EDT,2,hatssd,512 GB,129.99,US,139.99,1.0,139.99,139.99
10,263246989168,US $79.99,1,2019-06-12,2019-06-13 21:19:24,EDT,2,hatssd,256 GB,79.99,US,79.99,1.0,79.99,79.99
11,263246989168,US $79.99,1,2019-06-06,2019-06-13 11:35:59,EDT,2,hatssd,256 GB,79.99,US,79.99,1.0,79.99,79.99
12,263246989168,US $79.99,1,2019-06-03,2019-06-13 15:36:13,EDT,2,hatssd,256 GB,79.99,US,79.99,1.0,79.99,79.99
13,263246989168,US $79.99,1,2019-06-01,2019-06-13 20:10:59,EDT,2,hatssd,256 GB,79.99,US,79.99,1.0,79.99,79.99
17,263956219938,US $58.99,1,2019-06-05,2019-06-13 20:11:31,EDT,2,hatssd,128 GB,58.99,US,58.99,1.0,58.99,58.99
18,263956219938,US $62.09,1,2019-06-04,2019-06-13 12:20:39,EDT,2,hatssd,128 GB,58.99,US,62.09,1.0,62.09,62.09
19,263332366472,US $58.99,1,2019-06-05,2019-06-13 14:50:03,EDT,2,hatssd,128 GB,58.99,US,58.99,1.0,58.99,58.99
20,263332366472,US $58.99,1,2019-06-03,2019-06-13 19:12:29,EDT,2,hatssd,128 GB,58.99,US,58.99,1.0,58.99,58.99


In [258]:
outFile="HatssdID.csv"
projectFolder=r"D:\fledging\projects\mins\output\csv\may\20193105"
df_curr.loc[df_curr['seller']=="hatssd"].sort_values("id")["id"].to_csv(projectFolder+"\\"+outFile,header=False)

In [269]:
outFile="allID.csv"
projectFolder=r"D:\fledging\projects\mins\output\csv\may\20193105"
df_curr.sort_values("id")[["id","seller","revenue"]].to_csv(projectFolder+"\\"+outFile,header=False)

## FILTERS

NOT REQUIRED

In [117]:
#apr.groupby('capacity').capacity.nunique()
# np.random.seed(42)
# apr.loc[:,'cap2']=np.random.choice([128,250,256,500,512,1000],apr.shape[0])
# apr.groupby('cap2').id.nunique()

C:\Users\keths\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\keths\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


cap2
128     206
250     187
256     175
500     176
512     188
1000    192
Name: id, dtype: int64

In [71]:
sales_seller=df_curr.groupby(['seller']).agg({'revenue':"sum",
                        'quantity':"sum" } )

## ALL MARKET NUMBERS

In [72]:
top10_sales=sales_seller.nlargest(10,"revenue")[['revenue','quantity']]
top10_quant=sales_seller.nlargest(10,"quantity")[['revenue','quantity']]
print(top10_quant)
print(top10_sales)

                         revenue  quantity
seller                                    
hatssd                  19423.28       165
sellmac1                10998.90       110
edusolutions             7263.00        81
appleonus                4066.90        59
latency2050              5412.19        33
swingcomputers           3255.78        22
sbcorpsaving             2400.00        20
qbo12                    2204.81        19
illinoisprecisiongroup  11832.00        18
sta500                   2238.85        17
                         revenue  quantity
seller                                    
hatssd                  19423.28       165
illinoisprecisiongroup  11832.00        18
sellmac1                10998.90       110
edusolutions             7263.00        81
latency2050              5412.19        33
appleonus                4066.90        59
todbretton               3402.10        15
swingcomputers           3255.78        22
geektrusted              2713.20        16
sbcorpsavin

In [73]:
top10_sales=sales_seller.nlargest(10,"revenue")[['revenue','quantity']]
top10_quant=sales_seller.nlargest(10,"quantity")[['revenue','quantity']]
print(top10_quant)
print(top10_sales)

                         revenue  quantity
seller                                    
hatssd                  19423.28       165
sellmac1                10998.90       110
edusolutions             7263.00        81
appleonus                4066.90        59
latency2050              5412.19        33
swingcomputers           3255.78        22
sbcorpsaving             2400.00        20
qbo12                    2204.81        19
illinoisprecisiongroup  11832.00        18
sta500                   2238.85        17
                         revenue  quantity
seller                                    
hatssd                  19423.28       165
illinoisprecisiongroup  11832.00        18
sellmac1                10998.90       110
edusolutions             7263.00        81
latency2050              5412.19        33
appleonus                4066.90        59
todbretton               3402.10        15
swingcomputers           3255.78        22
geektrusted              2713.20        16
sbcorpsavin

In [74]:
top10_sales['FLAG']="revenue"
top10_quant['FLAG']="quantity"
mTop10=p.concat([top10_sales,top10_quant])

In [75]:
df_curr['seller'].nunique()

146

In [102]:
top10_merch=list(top10_sales.index.values)
df_curr['flag_seller']=np.where(np.isin(df_curr.loc[:,'seller'],'hatssd'),df_curr.loc[:,'seller'],"market")


In [103]:
df_curr['flag_seller2']=np.where(np.isin(df_curr.loc[:,'seller'],top10_merch),df_curr.loc[:,'seller'],"LeastTail")
mTop10Tail=p.concat([top10_sales,top10_quant])


In [109]:
df_curr.groupby(['flag_seller2']).agg({'revenue':"sum",
                        'quantity':"sum" } )

,revenue,quantity
flag_seller2,,
LeastTail,41798.5409,336
appleonus,4066.9000,59
edusolutions,7263.0000,81
geektrusted,2713.2000,16
hatssd,19423.2800,165
illinoisprecisiongroup,11832.0000,18
latency2050,5412.1900,33
sbcorpsaving,2400.0000,20
sellmac1,10998.9000,110


In [79]:
totalmarket_share=(df_curr.groupby(['flag_seller']).agg({'revenue':"sum",
                        'quantity':"sum" } )).sort_values('revenue',ascending=False)

In [80]:
marketcatXcapL2=df_curr.groupby(['cat','capacity']).agg({'revenue':"sum",
                        'quantity':"sum" } )
marketcatXcapL1=df_curr.groupby(['cat']).agg({'revenue':"sum",
                        'quantity':"sum" } )

In [81]:
marketcatXcapL1.index = [marketcatXcapL1.index.get_level_values(0).astype(str),
            ['Total'] * len(marketcatXcapL1)]
marketcatXcapL1.index

MultiIndex(levels=[['1', '2', '3', '4', '5'], ['Total']],
           codes=[[0, 1, 2, 3, 4], [0, 0, 0, 0, 0]],
           names=['cat', None])

In [82]:
marketcatXcapL2.index=[marketcatXcapL2.index.get_level_values(0).astype(str)
 ,marketcatXcapL2.index.get_level_values(1).astype(str)]
marketcatXcapL2.index

MultiIndex(levels=[['1', '2', '3', '4', '5'], [' 1 TB', ' 16 GB', ' 2 TB', ' 32 GB', ' 4 TB', ' 64 GB', '1  TB', '12  GB', '128 GB', '240 GB', '256 GB', '28  GB', '480 GB', '500 GB', '512 GB', '64  GB', '768 GB', 'NA', ' 1 TB', ' 2 TB', ' 64 GB']],
           codes=[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4], [0, 8, 9, 10, 11, 12, 15, 20, 5, 8, 10, 14, 17, 20, 0, 7, 8, 10, 14, 16, 18, 0, 1, 2, 3, 4, 6, 8, 10, 11, 13, 14, 17, 18, 19, 0, 8, 14]],
           names=['cat', 'capacity'])

In [83]:
marketcatXcapL2 = p.concat([marketcatXcapL2, marketcatXcapL1]).sort_index(level=[0,1])

In [84]:
marketTrend=df_curr.groupby(['date']).agg({'revenue':"sum",
                        'quantity':"sum" } )

In [85]:
marketcatXcapL2.head(20)

revenue  quantity
cat capacity                     
1    1 TB      449.7000         2
    128 GB     716.5900        19
    240 GB      56.2400         1
    256 GB     832.8700        15
    28  GB      70.3000         2
    480 GB     124.5000         1
    64  GB      45.2500         2
    Total     2325.4400        43
     64 GB      29.9900         1
2    64 GB      93.9800         2
    128 GB     836.9112        14
    256 GB    1312.6724        18
    512 GB    1435.8900         9
    NA          29.8800         1
    Total     3768.8448        46
     64 GB      59.5112         2
3    1 TB      304.9500         1
    12  GB     479.9600         4
    128 GB     360.7431         7
    256 GB     512.8900         8

In [86]:
df_curr.loc[df_curr['flag_seller']=='market'].groupby(['flag_seller']).agg({'revenue':"sum",
                        'quantity':"sum" } )

,revenue,quantity
flag_seller,,
market,93142.6109,710


## FLEDGING DATA

In [87]:
MarketTotal=df_curr.agg({
           'revenue':"sum",
           'quantity':"sum"
          } )
apr_fledging= df_curr.loc[df_curr['seller']=="hatssd"]
FledgingTotal=apr_fledging.agg({
           'revenue':"sum",
           'quantity':"sum"
          } )
all=p.DataFrame(data=None,index=["Fledging","Market"])
all.loc["Fledging","Revenue"]=FledgingTotal.revenue
all.loc["Market","Revenue"]=MarketTotal.revenue
all.loc["Fledging","Quantity"]=FledgingTotal.quantity
all.loc["Market","Quantity"]=MarketTotal.quantity
all

,Revenue,Quantity
Fledging,19423.2800,165.0
Market,112565.8909,875.0


In [89]:
marketTrendFledging=apr_fledging.groupby(['date']).agg({'revenue':"sum",
                        'quantity':"sum" })
marketcatXcapL2Fledging=apr_fledging.groupby(['cat','capacity']).agg({'revenue':"sum",
                        'quantity':"sum" } )

marketcatXcapL1Fledging=apr_fledging.groupby(['cat']).agg({'revenue':"sum",
                        'quantity':"sum" } )

marketcatXcapL1Fledging.index=marketcatXcapL1Fledging.index.astype(str)
marketcatXcapL2Fledging.index=[marketcatXcapL2Fledging.index.get_level_values(0).astype(str)
 ,marketcatXcapL2Fledging.index.get_level_values(1).astype(str)]

In [90]:
marketcatXcapL1Fledging.index = [marketcatXcapL1Fledging.index.get_level_values(0),
            ['Total'] * len(marketcatXcapL1Fledging)]
marketcatXcapL1Fledging.index

MultiIndex(levels=[['1', '2', '3', '4'], ['Total']],
           codes=[[0, 1, 2, 3], [0, 0, 0, 0]],
           names=['cat', None])

In [91]:
marketcatXcapL2Fledging = p.concat([marketcatXcapL2Fledging, marketcatXcapL1Fledging]).sort_index(level=[0,1])


In [92]:
marketcatXcapL2Fledging.index

MultiIndex(levels=[['1', '2', '3', '4'], [' 1 TB', '1  TB', '128 GB', '256 GB', '512 GB', 'Total', ' 1 TB', ' 2 TB']],
           codes=[[0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3], [0, 3, 5, 2, 3, 4, 5, 0, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 7]],
           names=['cat', 'capacity'])

In [93]:
mShare=marketcatXcapL2.merge(marketcatXcapL2Fledging,on=['cat','capacity'],how='inner',suffixes=('_MARKET','_FL'))

In [94]:
mShare['quant_share']=(mShare['quantity_FL']/mShare['quantity_MARKET']).map('{:,.5f}'.format)
mShare['rev_share']= (mShare['revenue_FL']/mShare['revenue_MARKET']).map('{:,.5f}'.format)

In [95]:
mShare.head(10)

revenue_MARKET  quantity_MARKET  revenue_FL  quantity_FL  \
cat capacity                                                             
1    1 TB           449.7000                2      449.70            2   
    256 GB          832.8700               15      207.00            3   
    Total          2325.4400               43      656.70            5   
2   128 GB          836.9112               14      534.01            9   
    256 GB         1312.6724               18      719.91            9   
    512 GB         1435.8900                9      399.97            3   
    Total          3768.8448               46     1653.89           21   
3    1 TB           304.9500                1      304.95            1   
    128 GB          360.7431                7      249.75            5   
    256 GB          512.8900                8      227.97            3   

             quant_share rev_share  
cat capacity                        
1    1 TB        1.00000   1.00000  
    256 GB       0.20000   0.24854  
    Total        0.11628   0.28240  
2   128 GB       0.64286   0.63807  
    256 GB       0.50000   0.54843  
    512 GB       0.33333   0.27855  
    Total        0.45652   0.43883  
3    1 TB        1.00000   1.00000  
    128 GB       0.71429   0.69232  
    256 GB       0.37500   0.44448

In [96]:
fledgingMarketTrend=apr_fledging.groupby(['date']).agg({'revenue':"sum",
                        'quantity':"sum" } )

## OUTPUT DATA SETS 

In [107]:
projectFolder=r"D:\fledging\projects\ebayMA\June\20190614"

In [98]:
outFile="marketTrend.csv"
marketTrend['type']="Market"
fledgingMarketTrend['type']="Fledging"
trend=p.concat([marketTrend,fledgingMarketTrend])
#trend.reindex(trend.type)
#trend['date'] = trend.index
trend.reset_index(level=0, inplace=True)
#to_csv(projectFolder+"\\"+outFile)
trend.to_csv(projectFolder+"\\"+outFile)

In [99]:
outFile="monthOall.csv"
all.to_csv(projectFolder+"\\"+outFile)

In [100]:
outFile="top10.csv"
mTop10.to_csv(projectFolder+"\\"+outFile)

In [101]:
outFile="mShare.csv"
mShare.to_csv(projectFolder+"\\"+outFile)

In [108]:
outFile="mTop10Tail.csv"
mTop10Tail.to_csv(projectFolder+"\\"+outFile)